# Introduction
Read a template notebook .py file and generate a main.js setup file for use with "Setup" nbextension

### Library Imports

In [ ]:
import pandas as pd
import numpy as np
import re
import os, sys
from pathlib import Path

### Jupyter-specific Imports and Settings

In [ ]:
# Data manipulation
# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Analysis/Modeling

In [ ]:
ignore = """
Types of lines

# !/usr/bin/env python
# coding: utf-8

Markdown: 
# # Title

start of code cell:  
# In[ ]:  OR  # In[12]:
# <python code lines>

#!/usr/bin/env python
# coding: utf-8
# # Title
# # Description
# In[ ]:
# # Environment
# ## Library Imports
# In[12]:
import pandas as pd
import numpy as np

"""

In [ ]:
js_preamble = """
define(['base/js/namespace', 'base/js/events'], function (Jupyter, events) {
  // Template cells including markdown and imports
  var setUp = function () {
"""
# The contents of the setUp function are insert_cell_at_index calls in between these two

js_postamble = """
    // Run all cells
    Jupyter.notebook.execute_all_cells()
  }
  // Prompts user to enter name for notebook
  var promptName = function () {
    // Open rename notebook box if 'Untitled' in name
    // if (Jupyter.notebook.notebook_name.search('Untitled') != -1) {
    //  document.getElementsByClassName('filename')[0].click()
    // }
  }
  // Run on start
  function load_ipython_extension () {
    // Add default cells for new notebook
    if (Jupyter.notebook.get_cells().length === 1) {
      setTimeout(setUp, 500)
    } else {
      promptName()
    }
  }
  // Run when cell is executed
  events.on('execute.CodeCell', function () {
    promptName()
  })
  // Run when notebook is saved
  events.on('before_save.Notebook', function () {
    promptName()
  })
  return {
    load_ipython_extension: load_ipython_extension
  }
})
"""

In [ ]:
# Jupyter.notebook.insert_cell_at_index('markdown', 1).set_text(`### Imports
#    Import libraries and write settings here.`)

def emit_cell_at_index(cell_type: str, inx: int, text: str):
    # cell_type is 'markdown' or 'code'
#     print(text)
    xs_fmt = 'Jupyter.notebook.insert_cell_at_index(\'{}\', {}).set_text(`{}`)\n'
    return xs_fmt.format(cell_type, inx, text)

def emit_cell(cell_type: str, cell_index: int, text: str):
    xs = None
    if True: #state != ParseState.none:
        #cell_type = 'markdown' if state == ParseState.markdown else 'code'
        xs = emit_cell_at_index(cell_type, cell_index, text)
        cell_index += 1
    return xs, cell_index

def trim_leading_and_trailing_lines(xs):
    # xs = '\n\n\n\nTest String\n\n\n\n\nSecond Line\n\nThird Line\n\n\n\n\n\n'
    trimmed = xs.split('\n')
    while trimmed and not trimmed[-1]:
        trimmed.pop()
    while trimmed and not trimmed[0]:
        trimmed.pop(0)
    return '\n'.join(trimmed)

In [ ]:
re_hdr1 = re.compile('^#!\/usr\/bin\/env.*$')
re_hdr2 = re.compile('^# coding: .*$')
re_markdown = re.compile('^# (#.*)$')
re_code_cell_start = re.compile('^# In\[[0-9 ]*\].*$')
re_blank_line = re.compile('^\s*$')

In [ ]:
def ignore_line(line):
    exact_matches = {
        '#!/usr/bin/env python',
        '# coding: utf-8',
        ''
    }
    return line in exact_matches

In [ ]:
def get_notebook_template_path(notebook_template_name: str) -> Path:
    # If your template notebook is called "notebook-template", then choosing
    # "Download As... Python (.py)" will save a file in the ~/Downloads folder:
    #    ~/Downloads/notebook-template.py.html
    path_fmt = '~/Downloads/{}.py.html'
    xpath = Path(path_fmt.format(notebook_template_name))
    return Path(os.path.expanduser(xpath))

# def get_nbextensions_path() -> Path:
#     # https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
#     rx = !{sys.executable} -m pip show jupyter_contrib_nbextensions
#     print(rx)
#     return rx.fields()['Location:']

def jupyter_contrib_nbextensions_slist_to_dict(slist):
    # slist is a list of lists
    # The first element of each list looks like a dictionary key
    # function named like this since it's only tested with the output from:
    #    rx = !{sys.executable} -m pip show jupyter_contrib_nbextensions
    #    jupyter_contrib_nbextensions_slist_to_dict(rx.fields())['Location']
    rx = {}
    for field in slist:
        key, val = field[0][:-1], field[1:]
        rx[key] = ' '.join(val)
    return rx

def get_sitepackages_path() -> Path:
    # https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
    rx = !{sys.executable} -m pip show jupyter_contrib_nbextensions
    return Path(jupyter_contrib_nbextensions_slist_to_dict(rx.fields())['Location'])

In [ ]:
get_sitepackages_path()

In [ ]:
rx

In [ ]:
from enum import Enum
class ParseState(Enum):
    none = 0
    markdown = 1
    codecell = 2

template_path = get_notebook_template_path('notebook-template')
state, prevstate = ParseState.none, ParseState.none

cell_index = 0
accum = ''
need_emit = False
setup_guts = ''
with open(template_path) as fp:
    for ix, line in enumerate(fp):
#         print('State: {}, accum: {}'.format(state, accum[:-1]))
        if re.search(re_hdr1, line) or re.search(re_hdr2, line):
            pass # print('header')
        else:
            mm = re.match(re_markdown, line)
            if mm:
                state, prevstate = ParseState.markdown, state
                if need_emit:
                    xs, cell_index = emit_cell(cell_type, cell_index, trim_leading_and_trailing_lines(accum))
                    setup_guts += xs
#                     print('Emitted: {}'.format(xs))
                    accum = ''
                cell_type = 'markdown'
                need_emit = True
                accum += mm.group(1)
            elif re.search(re_code_cell_start, line):
                state, prevstate = ParseState.codecell, state
                if need_emit:
                    xs, cell_index = emit_cell(cell_type, cell_index, trim_leading_and_trailing_lines(accum))
                    setup_guts += xs
#                     print('Emitted: {}'.format(xs))
                    accum = ''
                cell_type = 'code'
                need_emit = True
                # don't append codecellstart text to accum
            else:
                accum += line
                
#             print(xs)

    if need_emit:
        xs, cell_index = emit_cell(cell_type, cell_index, 
                                   trim_leading_and_trailing_lines(accum + line))
        setup_guts += xs
#         print('Emitted: {}'.format(xs))
        accum = ''

nbextsetuppath = get_sitepackages_path() / 'jupyter_contrib_nbextensions' / 'nbextensions'/ 'setup'
with open(nbextsetuppath / 'main.js', 'w') as fp:
    print(js_preamble + setup_guts + js_postamble, file=fp)

In [ ]:
# cp  -R notebook-template-generator/setup  /Users/john/development/Python/Virtualenvs/py37/lib/python3.7/site-packages/jupyter_contrib_nbextensions/nbextensions/  

In [ ]:
print(get_sitepackages_path() / 'jupyter_contrib_nbextensions' / 'nbextensions'/ 'setup')

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here

# References

In [ ]:
# Python
# https://stackoverflow.com/questions/11555468/how-should-i-read-a-file-line-by-line-in-python
# https://www.geeksforgeeks.org/python-string-length-len/
# https://docs.python.org/3/library/enum.html
# https://docs.python.org/3/library/re.html
# https://www.journaldev.com/23763/python-remove-spaces-from-string
# https://stackoverflow.com/questions/2504411/proper-indentation-for-python-multiline-strings
# https://docs.python.org/3/library/textwrap.html
# https://asoldatenko.com/can-i-copy-string-in-python-and-how.html
# https://stackoverflow.com/questions/5214578/python-print-string-to-text-file
# https://stackoverflow.com/questions/14225608/python-how-to-use-regex-in-an-if-statement
# https://medium.freecodecamp.org/how-to-quickly-find-type-issues-in-your-python-code-with-pytype-c022782f61c3

# Parsing
# https://tomassetti.me/antlr-mega-tutorial/
# https://tomassetti.me/parsing-in-python/
# https://rubular.com/

# JavaScript
# https://beautifier.io/
# http://www.javascriptlint.com/online_lint.php

# Koehrsen
# https://medium.com/search?q=koehrsen%20jupyter
# https://towardsdatascience.com/jupyter-notebook-extensions-517fa69d2231
# https://towardsdatascience.com/set-your-jupyter-notebook-up-right-with-this-extension-24921838a332
# https://github.com/WillKoehrsen/Data-Analysis
# https://towardsdatascience.com/how-to-write-a-jupyter-notebook-extension-a63f9578a38c
# https://towardsdatascience.com/how-to-automatically-import-your-favorite-libraries-into-ipython-or-a-jupyter-notebook-9c69d89aa343
# https://medium.com/@rrfd/cookiecutter-data-science-organize-your-projects-atom-and-jupyter-2be7862f487e

# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
# https://gist.github.com/parente/b6ee0efe141822dfa18b6feeda0a45e5
# https://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.text.html

# Unit Tests

In [ ]:
import unittest
        
class Test_My_Code(unittest.TestCase):

    def __init__(self, methodName='runTest'):
        # A new TestTest instance is created for each test method
        # Thus, __init__ is called once for each test method
        super(Test_My_Code, self).__init__(methodName)
#         print('__init__')
    
    def is_regex_match(regx, test: str, expected: str):
        mm = re.match(regx, test)
        if not mm:
            return False
        last_grp = len(mm.groups())
        return expected == mm.group(last_grp)
    
        # Regex tests
    regex_test_mls = """
    #!/usr/bin/env python
    # coding: utf-8
    # # Title
    # # Description
    # In[ ]:

    # # Environment
    # ## Library Imports
    # In[12]:
    import pandas as pd
    import numpy as np
    """
    
    def test_regex(self):
        self.assertTrue(is_regex_match(re_markdown, '# ## Library Imports', '## Library Imports'))

# Experiments

In [ ]:
1/0 # Stop here

In [ ]:
js_preamble

In [ ]:
emit_cell_at_index('markdown', 3, '# Analysis/Modeling')

In [ ]:
re_markdown = re.compile('^# (#.*)$')

xs = '# # Title'
mm = re.match(re_markdown, xs)
mm.group(1)

In [ ]:
xs

In [ ]:
xs = '\n\n\n\nTest String\n\n\n\n\nSecond Line\n\nThird Line\n\n\n\n\n\n'
trimmed = xs.split('\n')
while trimmed and not trimmed[-1]:
    trimmed.pop()
while trimmed and not trimmed[0]:
    trimmed.pop(0)
'\n'.join(trimmed)

In [ ]:
trimmed

In [ ]:
# Regex tests
_ = """
#!/usr/bin/env python
# coding: utf-8
# # Title
# # Description
# In[ ]:

# # Environment
# ## Library Imports
# In[12]:
import pandas as pd
import numpy as np
"""

In [ ]:
# jupyter contrib nbextensions install --user 
# pip show jupyter_contrib_nbextensions
# pip install --user jupyter_contrib_nbextensions && jupyter contrib nbextension install --user
# This:
# pip install  jupyter_contrib_nbextensions && jupyter contrib nbextension install --user

In [ ]:
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
rx = !{sys.executable} -m pip show jupyter_contrib_nbextensions

In [ ]:
rx

In [ ]:
type(rx)

In [ ]:
# https://gist.github.com/parente/b6ee0efe141822dfa18b6feeda0a45e5
rx.fields() #['Location:']

In [ ]:
get_notebook_template_path('notebook-template')

In [ ]:
524288/1024

In [ ]:
regex_test_mls = """
#!/usr/bin/env python
# coding: utf-8
# # Title
# # Description
# In[ ]:

# # Environment
# ## Library Imports
# In[12]:
import pandas as pd
import numpy as np
"""



In [ ]:
regex_test_mls.split('\n')

In [ ]:
re_hdr1 = re.compile('^#!\/usr\/bin\/env.*$')
re_hdr2 = re.compile('^# coding: .*$')
re_markdown = re.compile('^# (#.*)$')
re_code_cell_start = re.compile('^# In\[[0-9 ]*\].*$')
re_blank_line = re.compile('^\s*$')

In [ ]:
is_regex_match(re_hdr1, '#!/usr/bin/env python', '#!/usr/bin/env python')

In [ ]:
mm = re.match(re_hdr1, '#!/usr/bin/env python')
if mm is not None:
    mm.lastindex()

In [ ]:
mm

In [ ]:
len(mm.groups())

In [ ]:
mm = re.match(re_markdown, '# ## Library Imports')
len(mm.groups())


In [ ]:
!pwd

In [ ]:
rx = !{sys.executable} -m pip show jupyter_contrib_nbextensions

In [ ]:
rx